In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import cifar10
from keras.models import Sequential, Model 
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPool2D, BatchNormalization , Activation, ReLU
from keras.layers import Input, Dense, AveragePooling2D, GlobalAveragePooling2D
from keras import datasets, layers, models, losses
from keras.utils import np_utils
from keras import backend as K
import time
import cv2
import numpy as np
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from google.colab import drive
# import keras.backend as K

# 시각 전용
# from keract import get_activations, display_activations
drive.mount('/content/drive')
imgfile='drive/MyDrive/tensor/test1.png'

batch_size = 32
epochs = 3
num_classes = 10

# CIFAR-10 다운받은 데이터 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


# train 데이터, test 데이터 정규화
# 각각 load, 0~1 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0



model = Sequential()
model.add(Conv2D(64, 3, strides= 1, padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(64, 3, strides= 1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2,2), strides=2))

model.add(Conv2D(128, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(128, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Conv2D(256, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(256, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(256, 3, strides=1, padding='same'))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(MaxPool2D((2, 2), strides=2))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()



In [ ]:
batch_size = 64
epoch = 5
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])


log_dir = './logs/tensorflow' # 로그 
tensorboard_cb = keras.callbacks.TensorBoard(log_dir)

# CIFAR-10 모델 FIT / epochs 설정
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch, callbacks=[tensorboard_cb])
print("\n\nCIFAR-10 모델 학습 완료\n")

print("\nhistory의 key들")
print(history.history.keys())

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from matplotlib import cm
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import seaborn as sns


feat_extractor = Model(inputs=model.input,outputs=model.get_layer('dense_1').output) 
feat_extractor2 = Model(inputs=model.input,outputs=model.get_layer('dense_2').output)
features = feat_extractor.predict(x_test, batch_size=batch_size)
features2 = feat_extractor2.predict(x_test, batch_size=batch_size)

tsne = TSNE(n_components=2).fit_transform(features)
tsne2 = TSNE(n_components=2).fit_transform(features2)


In [ ]:

tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

df = pd.DataFrame()
df["y"] = y_test[:,0]
df["comp-1"]=tx
df["comp-2"]=ty

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="T-SNE projection")
%matplotlib inline
# K-means에 사용할 데이터 데이터프레임으로 만들기
cluster_df = pd.DataFrame(data=tsne, columns=['tx','ty'])
X_aniso = tsne
# 1.K-means 할당
k_means = KMeans(n_clusters=10, init='k-means++', max_iter=200, random_state=12)
    # X는 cluster_df의 feature array임.
cluster_labels = k_means.fit_predict(X_aniso)
k_means_labels = k_means.labels_
print('k_means_labels : ', k_means_labels)

k_means_cluster_centers = k_means.cluster_centers_
print('k_means_cluster_centers : ', k_means_cluster_centers)

In [ ]:
def clust_stats(cluster):
    class_freq = np.zeros(10)
    for i in range(10):
        class_freq[i] = np.count_nonzero(cluster == i)
    most_freq = np.argmax(class_freq)
    n_majority = np.max(class_freq)
    n_all = np.sum(class_freq)
    return (n_majority, n_all, most_freq)
  
def clusters_stats(predict, y):
    stats = np.zeros((10,3))
    for i in range(10):
        indices = np.where(predict == i)
        cluster = y[indices]
        stats[i,:] = clust_stats(cluster)
    return stats
  
def clusters_purity(clusters_stats):
    majority_sum  = clusters_stats[:,0].sum()
    n = clusters_stats[:,1].sum()
    return majority_sum / n

In [ ]:
def plot_d(digit, label):
    plt.axis('off')
    plt.imshow(digit.reshape((32,32,3)), cmap=plt.cm.gray)
    plt.title(label)

def plot_ds(digits, title, labels):
    n=digits.shape[0]
    n_rows=n/25+1
    n_cols=25
    plt.figure(figsize=(n_cols * 0.9, n_rows * 1.3))
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.suptitle(title)
    for i in range(n):
        plt.subplot(n_rows, n_cols, i + 1)
        plot_d(digits[i,:], "%d" % labels[i])
        
def plot_clusters(predict, y, stats):
    for i in range(10):
        indices = np.where(predict == i)
        title = "Most freq item %s, cluster size %d, majority %d, Label %d " % (label_encoding[int(stats[i,2])], stats[i,1], stats[i,0],stats[i,2])
        plot_ds(x_test[indices][:20], title, y[indices])




In [ ]:
from sklearn.mixture import GaussianMixture
def visualize_cluster_plot(clusterobj, dataframe, label_name, iscenter=True):
    
    # 군집별 중심 위치: K-Means, Mean Shift 등
    if iscenter:
        centers = clusterobj.cluster_centers_
    
    # Cluster 값 종류
    unique_labels = np.unique(dataframe[label_name].values)
    
    markers=['o', 's', '^', 'x', '*']
    isNoise=False

    for label in unique_labels:
        # 군집별 데이터 프레임
        label_cluster = dataframe[dataframe[label_name]==label]
        
        if label == -1:
            cluster_legend = 'Noise'
            isNoise=True
        else:
            cluster_legend = 'Cluster '+str(label)
        
        # 각 군집 시각화
        plt.scatter(x=label_cluster['tx'], y=label_cluster['ty'], s=10,
                    edgecolor='k', label=cluster_legend)
        
        # 군집별 중심 위치 시각화
        if iscenter:
            center_x_y = centers[label]
            plt.scatter(x=center_x_y[0], y=center_x_y[1], s=250, color='white',
                        alpha=0.9, edgecolor='k')
            plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k',\
                        edgecolor='k', marker='$%d$' % label)
    
    if isNoise:
        legend_loc='upper center'
    else: 
        legend_loc='upper right'
    
    plt.legend(loc=legend_loc)
    plt.figure(figsize=(10,10))
    plt.show()

In [ ]:
## 기본 tsne 시각화
from sklearn.manifold import TSNE
from matplotlib import cm
import matplotlib.pyplot as plt
from PIL import Image
width = 4000
height = 3000
max_dim = 100

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="T-SNE projection")

In [ ]:
# 지정된 크기로 초기화
fig = plt.figure(figsize=(6, 4))

# 레이블 수에 따라 색상 배열 생성, 고유한 색상을 얻기 위해 set(k_means_labels) 설정
colors = plt.cm.Spectral(np.linspace(0, 1, len(set(k_means_labels))))

# plot 생성
ax = fig.add_subplot(1, 1, 1)

for k, col in zip(range(10), colors):
    my_members = (k_means_labels == k)

    # 중심 정의
    cluster_center = k_means_cluster_centers[k]

    # 중심 그리기
    ax.plot(X_aniso[my_members, 0], X_aniso[my_members, 1], 'w', markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=6)

ax.set_title('K-Means')
ax.set_xticks(())
ax.set_yticks(())
plt.show()

In [ ]:

# GMM
gmm = GaussianMixture(n_components=10, random_state=0)
gmm_label = gmm.fit_predict(X_aniso)

# 군집 레이블 추가
cluster_df["gmm_label"] = gmm_label

# GMM은 cluster_centers_ 속성이 없음
visualize_cluster_plot(gmm, cluster_df, 'gmm_label',iscenter=False)

In [ ]:
stats = clusters_stats(predict, y_test)
purity = clusters_purity(stats)

print("Plotting an extract of the 10 clusters, overall purity: %f" % purity)

plot_clusters(predict, y_test, stats)

In [ ]:
stats = clusters_stats(gmm_label, y_test)
purity = clusters_purity(stats)

print("Plotting an extract of the 10 clusters, overall purity: %f" % purity)
plot_clusters(gmm_label, y_test, stats)